In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

In [4]:
# Load the data
pep = pd.read_csv('../results/16a_MSFragger_trypsin/peptide.tsv', sep='\t')
pep = pep[['Peptide','Protein', 'Mapped Proteins', 'Probability', 'Prev AA', 'Next AA']]
pep.columns = ['pep', 'main_protein', 'additional_proteins', 'prob', 'prev_aa','next_aa']
pep['pb_acc'] = pep['main_protein'].str.cat(pep['additional_proteins'], sep = ', ')
pep.pb_acc.fillna(pep.main_protein, inplace=True)
print(len(pep.index))
pep = pep[pep["pb_acc"].str.startswith("sp") == False]
print(len(pep.index))
pep['pb_acc'] = pep['pb_acc'].str.split(',')
accessions = pep['pb_acc']
mapped_proteins = []
for i in accessions:
    mapped_proteins.append([j.split('|')[1] for j in i if 'sp' not in j])
pep['pb_acc'] = mapped_proteins
pep = pep[['pep', 'pb_acc', 'prev_aa','next_aa']]
pep = pep.explode('pb_acc')
pep.head()

18432
18421
18421
18421


,pep,pb_acc,prev_aa,next_aa
0,AAAAAAAAAAGAAGGR,PABPN1-201,M,G
0,AAAAAAAAAAGAAGGR,PABPN1-202,M,G
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,SRP14-201,K,-
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,SRP14-204,K,-
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,ENST00000267884.11,K,-


In [45]:
pb_gene = pd.read_table('../results/1_make_gencode_database/gene_isoname.tsv')
print(pb_gene.head())
pb_gene = pd.Series(pb_gene.pr_gene.values, index=pb_gene.pb).to_dict()
gene_isoname = pd.read_table('../results/1_make_gencode_database/gene_isoname.tsv', names=['gene','isoname'])
gene_isoname = pd.Series(gene_isoname.gene.values, index=gene_isoname.isoname).to_dict()
gene_map = {**pb_gene, **gene_isoname}

pep =  pd.read_csv('/Users/tabea/mnt/xomics/tabear/pacbio/atlas_paper/lrp_cupcake/metamorpheus/search_results/Task1SearchTask/AllPeptides.psmtsv', sep='\t')
pep = pep[['Base Sequence','Protein Accession','Decoy/Contaminant/Target','QValue', 'Previous Amino Acid', 'Next Amino Acid']]
pep.columns = ['pep', 'pb_acc', 'targ', 'qval', 'prev_aa','next_aa']
# TODO add back in proper filter
# pep = pep[pep.targ == 'T']
pep = pep[(pep.targ =='T') & (pep.qval <= 0.01)]
# for now, all accessions
# TODO - decide how to deal with multiple PBs mapped to peptide
pep['pb_acc'] = pep['pb_acc'].str.split('|')
pep = pep[['pep', 'pb_acc', 'prev_aa','next_aa']]
pep = pep.explode('pb_acc')
pep['gene'] = pep['pb_acc'].map(gene_map)
pep.head()

       DDX11L1      DDX11L1-202
0      DDX11L1      DDX11L1-201
1       WASH7P       WASH7P-201
2    MIR6859-1    MIR6859-1-201
3  MIR1302-2HG  MIR1302-2HG-202
4  MIR1302-2HG  MIR1302-2HG-201


AttributeError: 'DataFrame' object has no attribute 'pr_gene'